In [12]:
library(tidyverse)
library(DESeq2)
library(sva)

source("../../utils/plots_eda.R")

# Load data

In [13]:
datasets <- c("GSE129508", "GSE58135", "GSE149276")

In [14]:
all_expression <- NULL
raw_expression <- NULL
all_metadata <- NULL

for(dataset in datasets){
    metadata <- read.table(paste0("before/", dataset, "/", dataset, ".sample_info.tsv"), header = TRUE, sep = "\t")
    expr_data <- read.table(paste0("before/", dataset, "/", dataset, ".counts.tsv"), header = TRUE, sep = "\t")


    # if sample GCSM3714613 is in expression or metadata, remove it
    if("GSM3714613" %in% colnames(expr_data)){
        print(paste0("Removing sample GSM3714613 from ", dataset))
        expr_data <- expr_data[, -which(colnames(expr_data) == "GSM3714613")]
        metadata <- metadata[metadata$sample_id != "GSM3714613",]
    }
    
    metadata$Dataset <- dataset
    metadata <- metadata %>% 
        mutate(Status = ifelse(Basal == 1, "Basal", "Luminal")) %>%
            # ifelse(is_LumA == 1, "LumA", "LumB"))) %>%
        mutate(Status = as.factor(Status))

    print(paste0("Samples: ", nrow(metadata), "; Features: ", nrow(expr_data)))

    #########################################################################################################################
    # Normalization # 
    # Create a DESeq2 dataset object
    expr_data <- expr_data %>% column_to_rownames("gene_id")

    # srop rows with only zeros
    expr_data <- expr_data[rowSums(expr_data) > 0,]

    dds <- DESeqDataSetFromMatrix(countData = expr_data, colData = metadata, design = ~ Status)
    # Normalize using median of ratios method
    dds <- estimateSizeFactors(dds)
    normalized_counts <- counts(dds, normalized = TRUE)
    norm_expr <- as.data.frame(normalized_counts) %>% 
        rownames_to_column("gene_id")
    print(paste0("Normalized Samples: ", nrow(metadata), "; Features: ", nrow(norm_expr)))

    #########################################################################################################################
    #  plot
    plot_res <- plot_diagnostic(norm_expr, metadata, dataset,
                                log_transform = FALSE, with_rowname = FALSE)
    layout <- (plot_res[[1]] + plot_res[[2]] ) / 
              (plot_res[[3]] )
    ggsave(paste0("before/", dataset, "/diagnostic_plot.png"), 
                plot = layout, width = 12, height = 12)

    # save data
    if(is.null(all_metadata)){
        all_metadata <- metadata
        all_expression <- norm_expr
        raw_expression <- expr_data %>% rownames_to_column("gene_id")
    } else {        
        all_metadata <- rbind(all_metadata, metadata)
        all_expression <- full_join(all_expression, norm_expr, by = "gene_id")
        raw_expression <- inner_join(raw_expression, expr_data %>% rownames_to_column("gene_id"), by = "gene_id")
    }
    print(paste0("Combined Samples: ", nrow(all_metadata), "; Features: ", nrow(all_expression)))
    print(" ")
}

# plot the combined data
print("Plotting combined data")
plot_res <- plot_diagnostic(all_expression, all_metadata, "Combined")
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
          (plot_res[[3]] )
ggsave("before/diagnostic_plot.png", 
            plot = layout, width = 12, height = 12)

[1] "Removing sample GSM3714613 from GSE129508"
[1] "Samples: 25; Features: 35238"
[1] "Normalized Samples: 25; Features: 30174"
[1] "..plotting.."


No id variables; using all as measure variables



[1] "Combined Samples: 25; Features: 30174"
[1] " "
[1] "Samples: 75; Features: 35238"
[1] "Normalized Samples: 75; Features: 34675"
[1] "..plotting.."


No id variables; using all as measure variables



[1] "Combined Samples: 100; Features: 34784"
[1] " "
[1] "Samples: 31; Features: 35238"
[1] "Normalized Samples: 31; Features: 31377"
[1] "..plotting.."


No id variables; using all as measure variables



[1] "Combined Samples: 131; Features: 34818"
[1] " "
[1] "Plotting combined data"
[1] "..plotting.."


No id variables; using all as measure variables

Warning message:
“Removed 68128 rows containing non-finite outside the scale range
(`stat_density()`).”
Warning message:
“Removed 68128 rows containing non-finite outside the scale range
(`stat_boxplot()`).”
Warning message:
“Removed 68128 rows containing non-finite outside the scale range
(`stat_summary()`).”


# Correction

In [15]:
all_metadata %>%
    group_by(Dataset, Status) %>% summarise(n())

`summarise()` has grouped output by 'Dataset'. You can override using the
`.groups` argument.


Dataset,Status,n()
<chr>,<fct>,<int>
GSE129508,Basal,4
GSE129508,Luminal,21
GSE149276,Basal,17
GSE149276,Luminal,14
GSE58135,Basal,36
GSE58135,Luminal,39


In [16]:
all_metadata %>%
    group_by(Dataset) %>% summarise(n())

Dataset,n()
<chr>,<int>
GSE129508,25
GSE149276,31
GSE58135,75


In [17]:
all_expression <- all_expression %>% as.data.frame() %>% column_to_rownames("gene_id")
raw_expression <- raw_expression %>% as.data.frame() %>% column_to_rownames("gene_id")

# sort index in all_expression_nona alphabetically - rows
all_expression <- all_expression[order(rownames(all_expression)),]
raw_expression <- raw_expression[order(rownames(raw_expression)),]


In [18]:
all_expression <- all_expression[, all_metadata$sample_id]
all_expression <- log2(all_expression + 1)
print(paste0("Number of samples: ", nrow(all_metadata)))
print(paste0("Number of features: ", nrow(all_expression), "; Number of samples: ", ncol(all_expression)))

[1] "Number of samples: 131"
[1] "Number of features: 34818; Number of samples: 131"


In [19]:
print(paste0("Number of features before filtering: ", nrow(all_expression)))
all_expression_nona <- all_expression[rownames(all_expression) %in% rownames(raw_expression),]
print(paste0("Number of features after filtering: ", nrow(all_expression_nona)))
print(paste0("Number of samples: ", nrow(all_metadata)))

[1] "Number of features before filtering: 34818"


[1] "Number of features after filtering: 28823"
[1] "Number of samples: 131"


In [20]:
design <- model.matrix(~all_metadata$Status)

corrected_expr <- sva::ComBat(dat = all_expression_nona, 
                              batch = all_metadata$Dataset, 
                              mod = design)

corrected_expr <- as.data.frame(corrected_expr)

# plot the combined corrected data
print("Plotting combined corrected data")
plot_res <- plot_diagnostic(corrected_expr, all_metadata, "Combined Corrected",
                            log_transform = TRUE, with_rowname = TRUE)
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
          (plot_res[[3]] )
ggsave("after/diagnostic_plot_corrected.png", 
            plot = layout, width = 12, height = 12)


Found3batches

Adjusting for1covariate(s) or covariate level(s)

Standardizing Data across genes

Fitting L/S model and finding priors

Finding parametric adjustments

Adjusting the Data




[1] "Plotting combined corrected data"
[1] "..plotting.."


No id variables; using all as measure variables



# Save data for correction and after correction

In [21]:
all_metadata$batch <- as.numeric(as.factor(all_metadata$Dataset)) - 1
# all_metadata$batch <- 0
all_metadata$lum <- as.numeric(as.factor(all_metadata$Status))
all_metadata$lum = all_metadata$lum - 1

for (dataset in unique(all_metadata$Dataset)) {
    print(paste0("Save data prior to batch correction for ", dataset))
    dataset_metadata <- all_metadata[all_metadata$Dataset == dataset,]
    dataset_metadata <- dataset_metadata %>% select(sample_id, lum, batch)
    
    dataset_expression <- all_expression[, dataset_metadata$sample_id]
    dataset_expression <- dataset_expression %>% rownames_to_column("gene_id")
    print(paste0("Samples: ", nrow(dataset_metadata), "; Features: ", nrow(dataset_expression)))
    
    write.table(dataset_metadata, 
        file = paste0("before/", dataset, "/design.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)
    write.table(dataset_expression, 
        file = paste0("before/", dataset, "/expr_for_correction.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)

}

write.table(all_metadata %>% select(sample_id, lum, batch), 
    file = "before/all_design.tsv", sep = "\t", quote = FALSE, row.names = FALSE)
write.table(all_expression %>% rownames_to_column("gene_id"),
    file = "before/all_expr_for_correction.tsv", sep = "\t", quote = FALSE, row.names = FALSE)
write.table(corrected_expr %>% rownames_to_column("gene_id"),
    file = "after/all_corrected_R_expr.tsv", sep = "\t", quote = FALSE, row.names = FALSE)

[1] "Save data prior to batch correction for GSE129508"
[1] "Samples: 25; Features: 34818"
[1] "Save data prior to batch correction for GSE58135"
[1] "Samples: 75; Features: 34818"
[1] "Save data prior to batch correction for GSE149276"
[1] "Samples: 31; Features: 34818"


# Small test data

In [22]:
small_meta <- all_metadata %>%
    # select samples - 3 from each lum class
    group_by(Dataset, lum) %>% sample_n(3) %>%
    ungroup() 

path_to_small <- "/home/yuliya/repos/cosybio/FedComBat/datasets/small_test/"

# set seed
set.seed(123)

# select 5 random rows (features) from the expression data
small_expr <- all_expression_nona[sample(1:nrow(all_expression_nona), 5), small_meta$sample_id]

write.table(small_meta,
      file = paste0(path_to_small, "/before/design.tsv"),
      sep = "\t", quote = FALSE)

write.table(small_expr,
      file = paste0(path_to_small, "/before/expr_for_correction.tsv"),
      sep = "\t", quote = FALSE)

for (dataset in unique(all_metadata$Dataset)) {
    print(paste0("Save data prior to batch correction for ", dataset))
    dataset_metadata <- small_meta[small_meta$Dataset == dataset,] %>%
        select(sample_id, lum, batch)
    
    dataset_expression <- small_expr[, dataset_metadata$sample_id]
    dataset_expression <- dataset_expression %>% rownames_to_column("gene_id")
    print(paste0("Samples: ", nrow(dataset_metadata), "; Features: ", nrow(dataset_expression)))
    
    write.table(dataset_metadata,
      file = paste0(path_to_small, "/before/", dataset, "/design.tsv"),
      sep = "\t", quote = FALSE, row.names = FALSE)
    write.table(dataset_expression, 
      file = paste0(path_to_small, "/before/",  dataset, "/expr_for_correction.tsv"),
      sep = "\t", quote = FALSE, row.names = FALSE)

}

design_small <- model.matrix(~small_meta$Status)
corrected_expr_small <- sva::ComBat(dat = small_expr, 
                              batch = small_meta$Dataset, 
                              mod = design_small) %>% as.data.frame()

write.table(small_expr,
      file = paste0(path_to_small, "/after/expr_central.tsv"),
      sep = "\t", quote = FALSE)


[1] "Save data prior to batch correction for GSE129508"
[1] "Samples: 6; Features: 5"
[1] "Save data prior to batch correction for GSE58135"
[1] "Samples: 6; Features: 5"
[1] "Save data prior to batch correction for GSE149276"
[1] "Samples: 6; Features: 5"


Found3batches

Adjusting for1covariate(s) or covariate level(s)

Standardizing Data across genes

Fitting L/S model and finding priors

Finding parametric adjustments

Adjusting the Data




# Fed Correction ...

... after FC app ...

In [24]:
path_to_fed <- "/home/yuliya/repos/cosybio/FedComBat/datasets/Breast_cancer_RNASeq/after/fed_res/"
zip_files <- list.files(path = path_to_fed, pattern = "\\.zip$", full.names = TRUE)

for(i in seq_along(zip_files)) {
  zipfile <- zip_files[i]
  zip_contents <- unzip(zipfile, list = TRUE)$Name
  csv_files <- zip_contents[grepl("\\.csv$", zip_contents, ignore.case = TRUE)]
  
  # If no CSV file is found, skip this zip
  if(length(csv_files) == 0) {
    warning(sprintf("No CSV file found in zip: %s", zipfile))
    next
  }
  csv_to_extract <- csv_files[1]
  new_csv_name <- file.path(path_to_fed, sprintf("%d_bayes_data.csv", i))
  unzip(zipfile, files = csv_to_extract, exdir = path_to_fed)
  
  # Determine the path to the extracted file (handles potential subdirectories)
  extracted_csv_path <- file.path(path_to_fed, csv_to_extract)
  
  # If the extracted file is inside a subdirectory, move it to the main folder
  if (!file.exists(extracted_csv_path)) {
    # Sometimes the unzip creates subfolders. List files recursively.
    extracted_files <- list.files(path_to_fed, pattern = "\\.csv$", full.names = TRUE, recursive = TRUE)
    # Find the matching file (by comparing filenames ignoring directory structure)
    candidate <- extracted_files[basename(extracted_files) == basename(csv_to_extract)]
    if(length(candidate) > 0) {
      extracted_csv_path <- candidate[1]
    } else {
      warning(sprintf("Extracted CSV file not found for zip: %s", zipfile))
      next
    }
  }
  
  # Rename (or move) the extracted CSV file to the new filename
  if(!file.rename(from = extracted_csv_path, to = new_csv_name)) {
    warning(sprintf("Failed to rename file: %s", extracted_csv_path))
  }
}


In [25]:
fed_expression <- NULL


for(i in 1:3){
    expr_data <- read.table(paste0(path_to_fed, i, "_bayes_data.csv"), header = TRUE, sep = "\t")
    print(paste0("Samples: ", ncol(expr_data), "; Features: ", nrow(expr_data)))
    # save data
    if(is.null(fed_expression)){
        fed_expression <- expr_data
    } else {        
        fed_expression <- full_join(fed_expression, expr_data, by = "gene_id")
    }
    print(paste0("Combined Samples: ", ncol(fed_expression), "; Features: ", nrow(fed_expression)))
    print(" ")
}


[1] "Samples: 76; Features: 28823"
[1] "Combined Samples: 76; Features: 28823"
[1] " "
[1] "Samples: 26; Features: 28823"
[1] "Combined Samples: 101; Features: 28823"
[1] " "
[1] "Samples: 32; Features: 28823"
[1] "Combined Samples: 132; Features: 28823"
[1] " "


In [26]:
fed_expression <- fed_expression %>% column_to_rownames("gene_id") 
fed_expression <- fed_expression[, all_metadata$sample_id]

# plot the combined data
print("Plotting combined data")
plot_res <- plot_diagnostic(fed_expression, all_metadata, "FedCombat corrected",
                            log_transform = TRUE, with_rowname = TRUE)
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
          (plot_res[[3]] )
ggsave("after/diagnostic_plot_correcter_Fed.png", 
            plot = layout, width = 12, height = 12)

[1] "Plotting combined data"


[1] "..plotting.."


No id variables; using all as measure variables



# Min Max and mean absolute error

In [27]:
print(paste0("Number of samples: ", nrow(all_metadata)))
print(paste0("Number of features FED: ", nrow(fed_expression), "; Number of samples: ", ncol(fed_expression)))
print(paste0("Number of features CENTRAL: ", nrow(corrected_expr), "; Number of samples: ", ncol(corrected_expr)))

[1] "Number of samples: 131"
[1] "Number of features FED: 28823; Number of samples: 131"
[1] "Number of features CENTRAL: 28823; Number of samples: 131"


In [28]:
corrected_expr <- as.data.frame(corrected_expr)
fed_expression <- fed_expression[rownames(corrected_expr), colnames(corrected_expr)]

In [29]:
# Calculate value-to-value mean, max, and mean absolute difference
mean_diff <- mean(as.matrix(abs(corrected_expr - fed_expression), na.rm = TRUE))
max_diff <- max(abs(corrected_expr - fed_expression), na.rm = TRUE)
min_diff <- min(abs(corrected_expr - fed_expression), na.rm = TRUE)

print(paste0("Min difference: ", min_diff))
print(paste0("Mean difference: ", mean_diff))
print(paste0("Max difference: ", max_diff))


[1] "Min difference: 5.82725316888855e-08"
[1] "Mean difference: 0.256842683799871"
[1] "Max difference: 11.3959829171489"


In [ ]:
boxplot(abs(as.numeric(unlist(corrected_expr)) - as.numeric(unlist(fed_expression))),
        main = "Boxplot of Differences", xlab = "Absolute Differences",
        # add values to the plot - for the mean and the median
        horizontal = TRUE)